In [3]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, load
import comet_ml
import os

# comet_ml_api_key = os.environ.get("COMET_ML_API_KEY")
api = comet_ml.api.API(
                       comet_ml_api_key
                       )


In [6]:
from comet_ml import Experiment
from comet_ml.api import API, APIExperiment
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, save
import pickle
import os

experiment = Experiment(
    api_key='',
    project_name="inflow-with-weather",
    workspace="drdevinhopkins",
)
api = API('')

set_log_level("ERROR")

target_column = 'Total Inflow hrly'

df = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns={target_column: 'y', 'Adm. requests cum': "Adm requests cum",
          'Pts.waiting for admission CUM': 'Pts waiting for admission CUM'}, inplace=True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

weather = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/weatherArchiveAndForecast.csv')
weather.ds = pd.to_datetime(weather.ds)

df = df.merge(weather, on='ds')

weather_regressors = ['temp', 'dew', 'humidity',
                      'precip', 'windspeed', 'sealevelpressure']

params = {
    # growth='off',
    'yearly_seasonality': False,
    'weekly_seasonality': True,
    'daily_seasonality': True,
    'n_lags': 4,
    'n_forecasts': 12,
    'changepoints_range': 0.95,
    'n_changepoints': 50,
    'quantiles': [0.2, 0.5, 0.8]
    # num_hidden_layers=4,
    # d_hidden=36,
    # learning_rate=0.005,
}

m = NeuralProphet(**params)
m = m.add_lagged_regressor(names=regressors)
for reg in weather_regressors:
    m = m.add_future_regressor(name=reg)
m = m.add_country_holidays("CA")
metrics = m.fit(df,
                freq='H',
                # progress='plot'
                )
print(metrics.tail(1))

experiment.log_parameters(params)
experiment.log_metrics(metrics.tail(1).iloc[0].to_dict())


COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/drdevinhopkins/inflow-with-weather/c5c9ef8074db45f19a2029fd05acc101
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Loss [61]  : (0.019538240507245064, 6.5257697105407715)
COMET INFO:     MAE [61]   : (2.4628653526306152, 67.0125503540039)
COMET INFO:     RMSE [61]  : (3.3178720474243164, 87.69439697265625)
COMET INFO:     RegLoss    : 0.0
COMET INFO:     epoch [61] : (0, 59)
COMET INFO:     hp_metric  : -1
COMET INFO:   Parameters:
COMET INFO:     changepoints_range : 0.95
COMET INFO:     daily_seasonality  :

Finding best initial lr:   0%|          | 0/260 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

         MAE      RMSE      Loss  RegLoss  epoch
79  2.388658  3.236181  0.017603      0.0     79


FileNotFoundError: [Errno 2] No such file or directory: '../models/inflow-with-weather.np'

In [7]:

save(m, "../../models/inflow-with-weather.np")

experiment.log_model("inflow-with-weather", "../../models/inflow-with-weather.np")


# with open('models/verticalTBS_forecast_model.pkl', "wb") as f:
#     pickle.dump(m, f)

experiment_name = experiment.name

experiment.end()


api.delete_registry_model('drdevinhopkins', 'inflow-with-weather')

apiexp = api.get('drdevinhopkins/inflow-with-weather/'+experiment_name)

apiexp.register_model('inflow-with-weather')

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/drdevinhopkins/inflow-with-weather/85ba5a51c224445f89463f6a41431dc2
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Loss [81]  : (0.017603032290935516, 5.853559970855713)
COMET INFO:     MAE [81]   : (2.388549327850342, 86.00037384033203)
COMET INFO:     RMSE [81]  : (3.2361812591552734, 108.73719024658203)
COMET INFO:     RegLoss    : 0.0
COMET INFO:     epoch [81] : (0, 79)
COMET INFO:     hp_metric  : -1
COMET INFO:   Parameters:
COMET INFO:     changepoints_range : 0.95
COMET INFO:     daily_seasonality  : True
COMET INFO:     n_changepoints     : 50
COMET INFO:     n_forecasts        : 12
COMET INFO:     n_lags             : 4
COMET INFO:     quantiles          : [0.2, 0.5, 0.8]
COMET INFO:     weekly_seasonality : True
COMET INFO:

{'registryModelId': 'EzsiKT7X03orqZc5k2XMFVr7Z',
 'registryModelItemId': 'laXUNqO15cVQUW82yT7K93EtE'}

In [8]:

# api.download_registry_model("drdevinhopkins", "inflow-with-weather",
#                             version="1.0.0", output_path="../comet-ml-models", expand=True, stage=None)

loaded_model = load("../../models/inflow-with-weather.np")

set_log_level("ERROR")


In [9]:

target_column = 'Total Inflow hrly'
df = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns={target_column: 'y', 'Adm. requests cum': "Adm requests cum",
          'Pts.waiting for admission CUM': 'Pts waiting for admission CUM'}, inplace=True)
df.dropna(inplace=True)
df = df.reset_index(drop=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

weather = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/weatherArchiveAndForecast.csv')
weather.ds = pd.to_datetime(weather.ds)
weather = weather.sort_values(by='ds', ascending=True)

last_timestamp = df.loc[len(df)-1].ds

df = df.merge(weather, on='ds')
# df['ID'] = 'test'

future = pd.concat(
    [df.tail(48), weather[weather.ds > last_timestamp].head(12)])
# future['ID'] = 'test'

# future = loaded_model.make_future_dataframe(df, periods=12) # periods=m.n_forecasts, n_historic_predictions=False

# forecast = loaded_model.predict(future, raw=True, decompose=False)

# start = forecast.values.tolist()[0][0]
# forecast_length = len(forecast.values.tolist()[0][1:])

# forecast_output = pd.DataFrame(columns=['ds','inflow','timestamp'])
# forecast_output['ds'] = pd.date_range(start=start, periods=forecast_length, freq='H')
# forecast_output['inflow'] = forecast.values.tolist()[0][1:]
# forecast_output['timestamp'] = pd.Timestamp.now().round('S').replace(tzinfo=None)
# forecast_output.to_csv('forecasts/inflow_with_weather.csv', index=False)


forecast = loaded_model.predict(future,
                                # raw=False,
                                # decompose=False
                                )

forecast_trimmed = forecast[forecast.y.isnull()].set_index('ds')
forecast_trimmed.to_csv('../../forecasts/inflow_with_weather_raw.csv')

start = forecast_trimmed.index.tolist()[0]
forecast_length = len(forecast_trimmed.index.tolist())
i = 0
timestamp = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output_list = []
for ds in pd.date_range(start=start, periods=forecast_length, freq='H'):
    i = i+1
    forecast_output_list.append(
        {'ds': ds, 'inflow': forecast_trimmed.loc[ds]['yhat'+str(i)], 'timestamp': timestamp})
forecast_output = pd.DataFrame(forecast_output_list)
forecast_output.to_csv('../../forecasts/inflow_with_weather.csv', index=False)


Predicting: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/np2/lib/python3.8/site-packages/neuralprophet/forecaster.py:3108: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_forecast[name] = yhat

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/np2/lib/python3.8/site-packages/neuralprophet/forecaster.py:3108: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_forecast[name] = yhat

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/np2/lib/python3.8/site-packages/neur